In [1]:
import pandas as pd
import numpy as np
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from scripts.preprocess_cpv import process_cpv_data
from scripts.preprocess_categorielle import preprocess_categorielle, encoding_idAccordCadre
from scripts.preprocess_missing_values import clean_missing_values

In [2]:
df = pd.read_csv('../data/train.csv', encoding='utf-8')

In [3]:
df = clean_missing_values(df)
df.isna().sum()

Remaining missing values:
dureeMois                0
marcheInnovant           0
tauxAvance               0
offresRecues             0
sousTraitanceDeclaree    0
tauxAvance               0
origineFrance            0
dtype: int64


nature                           0
montant                          0
codeCPV                          0
procedure                        0
dureeMois                        0
formePrix                        0
offresRecues                     0
marcheInnovant                   0
ccag                             0
sousTraitanceDeclaree            0
typeGroupementOperateurs         0
tauxAvance                       0
origineFrance                    0
idAccordCadre               187389
dtype: int64

In [4]:
# ajouter le cas CPV avec le code de Ronan et Paul


In [7]:
df = encoding_idAccordCadre(df)

In [8]:
columns_categorical = ['nature', 'procedure', 'formePrix', 'marcheInnovant', 'ccag', 'sousTraitanceDeclaree', 'typeGroupementOperateurs', 'origineFrance', 'idAccordCadre']
columns_numerical = ['montant', 'dureeMois', 'offresRecues', 'tauxAvance']
df_test = preprocess_categorielle(df, columns_categorical)

In [9]:
df_test

,montant,codeCPV,dureeMois,offresRecues,tauxAvance,nature_Marché,nature_Marché de défense ou de sécurité,nature_Marché de partenariat,procedure_Appel d'offres ouvert,procedure_Appel d'offres restreint,...,origineFrance_0.9,origineFrance_0.912,origineFrance_0.92,origineFrance_0.94,origineFrance_0.95,origineFrance_0.98,origineFrance_0.99,origineFrance_1.0,idAccordCadre_0,idAccordCadre_1
0,106866.59,45111291,23.0,2.0,0.05,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,6000000.00,45231400-9,36.0,1.0,0.00,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,2200000.00,45232410-9,12.0,2.0,0.00,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,2400000.00,71351810-4,48.0,1.0,0.00,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,2268857.60,33000000-0,36.0,1.0,0.00,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239725,50400.00,72500000-0,12.0,1.0,0.00,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
239726,440742.36,45262660-5,9.0,1.0,0.00,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
239727,77221.40,45421000-4,16.0,1.0,0.00,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
239728,212125.00,45262512,24.0,5.0,0.05,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
